In [194]:
import pandas as pd
import datetime

In [217]:
df = pd.read_csv('../../hank/Transformed_Minedex_data.csv')
df.head()

SiteCode   StartDate     EndDate  Tonnage (Mt)  Grade   GradeUnit  \
0  S0000001  13/07/2005  14/09/2006         0.695   1.30  gram/tonne   
1  S0000002  12/03/2019         NaN         0.528   2.09  gram/tonne   
2  S0000005  30/09/2006  07/03/2016         0.011   7.33  gram/tonne   
3  S0000005  30/09/2006  07/03/2016         0.018   4.94  gram/tonne   
4  S0000006  08/10/1993  30/12/2014         0.057   2.36  gram/tonne   

  CommodityName  ContainedMetal ContainedMetalUnit  CutOffGrade  \
0          Gold          903.50           kilogram          NaN   
1          Gold         1103.52           kilogram          0.3   
2          Gold           80.63           kilogram          NaN   
3          Gold           88.92           kilogram          NaN   
4          Gold          134.52           kilogram          NaN   

  TargetCommodityGroup  quantity(tonne)   ppm  
0                 GOLD          0.90350  1.30  
1                 GOLD          1.10352  2.09  
2                 GOLD          0.08063  7.33  
3                 GOLD          0.08892  4.94  
4                 GOLD          0.13452  2.36

In [218]:
df[df['SiteCode'] == 'S0001179']

SiteCode   StartDate EndDate  Tonnage (Mt)   Grade   GradeUnit  \
1039  S0001179  01/01/1978     NaN          0.45  100.00  gram/tonne   
5676  S0001179  01/01/1978     NaN          0.45    1.25     percent   
5677  S0001179  01/01/1978     NaN          0.45    1.50     percent   

     CommodityName  ContainedMetal ContainedMetalUnit  CutOffGrade  \
1039        Silver       45000.000           kilogram          NaN   
5676          Zinc           5.625          kilotonne          NaN   
5677          Lead           6.750          kilotonne          NaN   

      TargetCommodityGroup  quantity(tonne)      ppm  
1039  COPPER - LEAD - ZINC             45.0    100.0  
5676  COPPER - LEAD - ZINC           5625.0  12500.0  
5677  COPPER - LEAD - ZINC           6750.0  15000.0

In [196]:
df.shape

(17616, 13)

In [197]:
df['StartDate'].describe()

count          17616
unique          1428
top       30/06/2014
freq             755
Name: StartDate, dtype: object

In [198]:
df['EndDate'].describe()

count           7428
unique           788
top       29/06/2015
freq             463
Name: EndDate, dtype: object

### Get Most Recent Date for each site
- if contains EndDate, get most recent.
- if not get most recent StartDate

In [199]:
df_no_edate = df[df['EndDate'].isna()]
df_edate = df[~df['EndDate'].isna()]

In [200]:
df_edate.shape[0], df_edate.SiteCode.nunique(), df_no_edate.shape[0], df_no_edate.SiteCode.nunique()

(7428, 2154, 10188, 2365)

In [201]:
df_edate.shape[0] + df_no_edate.shape[0]

17616

In [202]:
7428 + 10188

17616

In [203]:
df_edate = df_edate.sort_values('EndDate').groupby('SiteCode').tail(1)

df_edate = df_edate.drop(columns=['StartDate'])
df_edate = df_edate.rename(columns={'EndDate': 'Date'})
df_edate['isStartDate'] = False

df_no_edate = df_no_edate.sort_values('StartDate').groupby('SiteCode').tail(1)

df_no_edate = df_no_edate.drop(columns=['EndDate'])
df_no_edate = df_no_edate.rename(columns={'StartDate': 'Date'})
df_no_edate['isStartDate'] = True

In [204]:
df_edate.shape, df_no_edate.shape

((2154, 13), (2365, 13))

In [205]:
df_both = pd.concat([df_edate, df_no_edate], axis=0)
df_both.shape

(4519, 13)

### Duplicate SiteCodes still exist??

In [206]:
df_both = df_both.sort_values('Date').groupby('SiteCode').tail(1)

In [207]:
df_both.shape

(4475, 13)

#### Create input dataframe

In [208]:
df_input = df_both[['SiteCode', 'Date', 'CommodityName', 'ppm']]
df_input

SiteCode        Date  CommodityName       ppm
17501  S0000991  01/01/1901  Uranium oxide     780.0
7213   S0001801  01/01/1958         Gypsum  900000.0
8098   S0003569  01/01/1958         Gypsum  890000.0
7210   S0001796  01/01/1958         Gypsum  800000.0
8091   S0003561  01/01/1958         Gypsum  850000.0
...         ...         ...            ...       ...
15717  S0230310  31/12/2020           Iron  573000.0
3307   S0016653  31/12/2020           Gold       2.9
13514  S0223726  31/12/2020           Iron  625000.0
3093   S0006504  31/12/2020           Gold       0.9
4250   S0022447  31/12/2020           Gold       1.2

[4475 rows x 4 columns]

### SiteMineralisation

In [211]:
df_mi = pd.read_csv('../../data/minedex/SiteMineralization.csv')

orth_deposits = ['Orthomagmatic mafic and ultramafic - layered mafic intrusions', \
        'Orthomagmatic mafic and ultramafic - komatiitic or dunitic']

df_mi = df_mi[df_mi['MineralizationStyle'].isin(orth_deposits)]

###### Merge

In [213]:
df_input = df_input.merge(df_mi[['SiteCode', 'MineralizationStyle']])
df_input.head()

SiteCode        Date CommodityName       ppm  \
0  S0001442  02/02/2006        Nickel  17900.00   
1  S0005584  02/02/2006        Cobalt    400.00   
2  S0032888  04/03/2019     Palladium      0.65   
3  S0231143  05/02/2016        Nickel  17000.00   
4  S0230972  05/02/2016        Nickel  29000.00   

                                 MineralizationStyle  
0  Orthomagmatic mafic and ultramafic - layered m...  
1  Orthomagmatic mafic and ultramafic - layered m...  
2  Orthomagmatic mafic and ultramafic - layered m...  
3  Orthomagmatic mafic and ultramafic - komatiiti...  
4  Orthomagmatic mafic and ultramafic - komatiiti...

In [214]:
df_input.shape

(54, 5)

### Too many NaNs :(

In [216]:
df_input.pivot(index='SiteCode', columns='CommodityName', values='ppm')

CommodityName  Aggregate - undifferentiated  Chromium oxide  Cobalt  Copper  \
SiteCode                                                                      
S0001197                                NaN             NaN     NaN     NaN   
S0001204                                NaN             NaN     NaN  4000.0   
S0001221                                NaN             NaN     NaN     NaN   
S0001273                                NaN             NaN     NaN     NaN   
S0001440                                NaN             NaN     NaN     NaN   
S0001442                                NaN             NaN     NaN     NaN   
S0001451                                NaN             NaN     NaN   400.0   
S0001584                                NaN             NaN     NaN     NaN   
S0001585                                NaN             NaN     NaN     NaN   
S0001667                                NaN             NaN     NaN     NaN   
S0001680                                NaN             NaN     NaN     NaN   
S0001757                                NaN             NaN     NaN     NaN   
S0001759                                NaN             NaN     NaN     NaN   
S0001764                                NaN             NaN     NaN     NaN   
S0001768                                NaN             NaN     NaN     NaN   
S0001769                                NaN             NaN     NaN     NaN   
S0001805                                NaN             NaN     NaN     NaN   
S0001806                                NaN             NaN     NaN     NaN   
S0002888                                NaN             NaN     NaN     NaN   
S0002890                                NaN             NaN     NaN     NaN   
S0003377                                NaN             NaN     NaN  4900.0   
S0003378                                NaN             NaN     NaN     NaN   
S0003500                          1000000.0             NaN     NaN     NaN   
S0003690                                NaN             NaN     NaN     NaN   
S0004074                                NaN             NaN     NaN     NaN   
S0004075                                NaN             NaN     NaN     NaN   
S0004076                                NaN             NaN     NaN     NaN   
S0004091                                NaN             NaN     NaN     NaN   
S0005584                                NaN             NaN   400.0     NaN   
S0006421                                NaN             NaN     NaN     NaN   
S0006422                                NaN             NaN     NaN     NaN   
S0006423                                NaN             NaN     NaN     NaN   
S0006755                                NaN             NaN     NaN     NaN   
S0016658                                NaN             NaN     NaN     NaN   
S0018904                                NaN             NaN     NaN     NaN   
S0019102                                NaN             NaN     NaN     NaN   
S0019502                                NaN             NaN     NaN     NaN   
S0020422                                NaN             NaN     NaN     NaN   
S0020474                                NaN             NaN     NaN     NaN   
S0021367                                NaN             NaN     NaN     NaN   
S0025535                                NaN             NaN     NaN     NaN   
S0032888                                NaN             NaN     NaN     NaN   
S0223588                                NaN             NaN     NaN     NaN   
S0223589                                NaN             NaN     NaN     NaN   
S0223611                                NaN             NaN     NaN     NaN   
S0224398                                NaN             NaN     NaN     NaN   
S0224815                                NaN        372000.0     NaN     NaN   
S0229528                                NaN             NaN     NaN     NaN   
S0229862                                NaN       

# END

### Filter by Abundant Minerals

In [ ]:
abdundant_minerals = ['Nickel', 'Iron']

In [ ]:
df_input = df_input[df_input['CommodityName'].isin(abdundant_minerals)]
df_input.shape

#### Pivot by SiteCode

In [ ]:
df_input = df_input.pivot(index='SiteCode', columns='CommodityName', values='ppm')
df_input

In [ ]:
df_input = df_input.rename_axis(None)
df_input

In [ ]:
df_input.isna().sum()

In [ ]:
df_input

In [ ]:
df_input.dropna()